# ASSIGNMENT:::
As mentioned earlier during class, now that you all have submitted your “semi-individual” assignments, it is time to get together in groups and figure out how to “trim down” the various exercises into something that you would like to submit as your portfolio for the oral exam.

 
During the exam, we will use the submission for additional questions. This means you have the opportunity to actively use your submission to influence the agenda for the exam.
 

We do not want you to simply take the 4 weekly assignments and concatenate them into a long notebook or multiple files in a folder/repository. What we want is for you to think about how you can make something creative and useful from the various things you have worked on, while you have been improving and learning. For instance, if you decide to submit a notebook with a regression supported by some good visualizations, that would be just fine. It’s not the quantity we are after.
The submission should hovewer have multiple notebooks for the different parts (please do not do superlong messy notebooks with endless outputs) and at least one Streamlit app (deployed! - and working) demonstrating the functionality. 
 

Also, the submissions are not graded directly.
 

Feel free to ask here if you have further questions.


# **M1 Exam Submission**

 * Business Problem: 

 * Objectives:

## **Table of Contents**

#### I. Import Libraries and Data
#### II. Data Preparation
#### III. Model Development
#### IV. Model Evaluation

In [1]:
print(123)
print("test")

123
test


In [1]:
print(1234)

1234
